In [4]:
import pandas as pd
import requests
import json
from config import api_key 
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress


In [14]:
#Load CSV Files with movie data
imdb_file = pd.read_csv("Resource/IMDb_All_Genres_etf_clean1.csv")
imdb_clean = imdb_file.drop(columns = ["Censor", "main_genre", "side_genre", "Runtime(Mins)"])
imdb_clean = imdb_clean.rename(columns = {"Rating": "IMDB_Rating"})
imdb_clean.head()

,Movie_Title,Year,Director,Actors,IMDB_Rating,Total_Gross
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,Gross Unkown
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,$534.86M
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,$377.85M
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,$292.58M
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,$342.55M


In [18]:
#Load CSV Files with movie data
metadata_file = pd.read_csv("Resource/movie_metadata.csv")
metadata_clean = metadata_file.drop(columns = ["color", "num_critic_for_reviews", "duration", "director_facebook_likes", 
                             "actor_3_facebook_likes", "actor_1_facebook_likes", "genres", "num_voted_users", 
                             "cast_total_facebook_likes", "facenumber_in_poster", "plot_keywords", "movie_imdb_link", 
                             "num_user_for_reviews", "language", "country", "content_rating", "aspect_ratio", "movie_facebook_likes",
                                              "actor_2_facebook_likes"])
metadata_clean.columns

Index(['director_name', 'actor_2_name', 'gross', 'actor_1_name', 'movie_title',
       'actor_3_name', 'budget', 'title_year', 'imdb_score'],
      dtype='object')

In [21]:
#Load CSV Files with movie data
top1000_file = pd.read_csv("Resource/Top_1000_Highest_Grossing_Movies_Of_All_Time.csv")
top1000_file = top1000_file.drop(columns = ["Genre", "Movie Rating", "Duration", "Votes", "Logline"])
top1000_file.head()

,Movie Title,Year of Realease,Gross,Worldwide LT Gross,Metascore
0,Avatar,2009,$760.51M,"$2,847,397,339",83
1,Avengers: Endgame,2019,$858.37M,"$2,797,501,328",78
2,Titanic,1997,$659.33M,"$2,201,647,264",75
3,Star Wars: Episode VII - The Force Awakens,2015,$936.66M,"$2,069,521,700",80
4,Avengers: Infinity War,2018,$678.82M,"$2,048,359,754",68
